In [120]:
import os
import numpy as np
import pandas as pd
import re

from utils.logger import LOGPATH

In [121]:
fpath = LOGPATH.joinpath('summary-fb.csv')
tori = pd.read_csv(fpath, engine='python')
tori

,seed,data,model,conv,epoch_best,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1macro_train,f1micro_train,f1macro_val,f1micro_val,f1macro_test,f1micro_test,time_eval,mem_ram_eval,mem_cuda_eval
0,20,cora,Iterative,AdjConv,63,500,4.1765,1.080,0.059,92.248,93.169,88.792,90.037,87.530,89.094,0.0011,1.080,0.059
1,21,cora,Iterative,AdjConv,45,500,4.0485,1.128,0.059,75.728,85.231,73.496,83.764,72.777,81.331,0.0011,1.128,0.059
2,22,cora,Iterative,AdjConv,50,500,4.0349,1.132,0.059,76.519,85.908,70.762,79.705,76.251,86.876,0.0011,1.132,0.059
3,20,cora,Iterative,ChebConv,47,500,6.4711,1.100,0.071,99.386,99.508,69.611,73.432,73.348,75.416,0.0017,1.100,0.071
4,21,cora,Iterative,ChebConv,46,500,6.2731,1.147,0.071,98.865,99.077,72.122,75.461,73.184,75.231,0.0017,1.147,0.071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,21,roman_empire,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",327,500,14.1943,1.151,0.635,45.778,54.001,42.922,50.827,41.352,49.834,0.0077,1.151,0.635
596,22,roman_empire,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",470,500,14.1967,1.172,0.635,44.940,53.273,41.533,49.945,40.087,49.018,0.0067,1.172,0.635
597,20,roman_empire,DecoupledFixedCompose,"AdjDiffConv,AdjDiffConv-appr,appr",347,500,9.2672,1.075,0.366,46.826,65.127,45.282,63.733,44.960,64.026,0.0057,1.075,0.366
598,21,roman_empire,DecoupledFixedCompose,"AdjDiffConv,AdjDiffConv-appr,appr",262,500,8.3608,1.162,0.366,47.674,65.407,46.886,64.284,45.645,64.335,0.0061,1.162,0.366


In [122]:
dlst = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['epoch_learn', 'time_learn', 'mem_ram_train', 'mem_cuda_train', 'f1micro_test', 'time_eval',]
t = tori[tori['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
t.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 3)

,seed,data,model,conv,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1micro_test,time_eval
557,22,flickr,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,5.7232,1.173,0.147,26.842,0.0020
556,21,flickr,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,5.6911,1.146,0.147,26.447,0.0019
555,20,flickr,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,6.8945,1.061,0.147,24.474,0.0019
548,22,pubmed,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,9.6916,1.187,0.334,83.359,0.0042
547,21,pubmed,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,10.1231,1.183,0.334,84.450,0.0036
546,20,pubmed,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,11.1456,1.064,0.334,83.511,0.0044
539,22,citeseer,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,3.8818,1.220,0.115,74.962,0.0022
538,21,citeseer,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,3.5475,1.216,0.115,74.962,0.0021
537,20,citeseer,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,7.2217,1.081,0.115,74.198,0.0022
530,22,cora,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",500,6.0491,1.109,0.070,88.909,0.0031


In [128]:
conv_repr = {
    'DecoupledFixed': {
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',
    },
    'DecoupledVar': {
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ChebConv2':        'ChebInterp',
        'ClenhawConv':      'Clenhaw',
        'BernConv':         'Bernstein',
    },
    'AdaGNN': {'AdaConv':   'AdaGNN'},
    'ACMGNN': {
        # 'ACMConv-1.0-low-high':     'FBGNNI',
        # 'ACMConv-2.0-low-high':     'FBGNNII',
        # 'ACMConv-1.0-low-high-id':  'ACMGNNI',
        # 'ACMConv-2.0-low-high-id':  'ACMGNNII',
        'ACMConv-2.0-low-high-id':  'ACMGNN',
    },
    'DecoupledFixedCompose': {
        'AdjiConv,AdjiConv-mono,mono': 'FAGNN',
        'Adji2Conv,Adji2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',
    },
    'DecoupledVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',
    }
}
flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

df = t.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df['time_learn_mean'] = df['time_learn_mean'] * 1000 / df['epoch_learn_mean']
df['time_learn_std'] = df['time_learn_std'] * 1000 / df['epoch_learn_mean']
df.drop(columns=['epoch_learn_mean', 'epoch_learn_std'], inplace=True)
df.reset_index(inplace=True)

df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
df.dropna(subset=['name'], inplace=True)
df.drop(columns=['model', 'conv'], inplace=True)
df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
df

,data,name,time_learn_mean,time_learn_std,mem_ram_train_mean,mem_ram_train_std,mem_cuda_train_mean,mem_cuda_train_std,f1micro_test_mean,f1micro_test_std,time_eval_mean,time_eval_std
3,actor,ACMGNN,42.606200,1.100886,1.230000,0.058847,0.413,0.0,32.675667,2.957597,0.007400,0.000265
4,actor,AdaGNN,10.009533,1.578104,1.170667,0.049075,0.141,0.0,38.070333,0.833196,0.001467,0.000115
5,actor,PPR,9.703267,0.443809,1.130667,0.049075,0.108,0.0,37.236667,0.840125,0.001300,0.000173
6,actor,Gaussian,9.855933,1.504255,1.151667,0.057274,0.109,0.0,37.763333,0.174053,0.001400,0.000200
7,actor,HK,9.421800,1.131418,1.120000,0.050269,0.108,0.0,35.679667,0.665349,0.001100,0.000173
...,...,...,...,...,...,...,...,...,...,...,...,...
169,squirrel_filtered,Chebyshev,9.255467,1.081134,1.137333,0.036019,0.072,0.0,37.086667,2.491115,0.001533,0.000058
170,squirrel_filtered,ChebInterp,34.821667,3.078537,1.149667,0.034385,0.073,0.0,36.711667,1.406646,0.006433,0.000058
171,squirrel_filtered,Clenhaw,12.782000,0.840600,1.139000,0.036373,0.072,0.0,32.732667,2.939555,0.001867,0.000058
172,squirrel_filtered,Horner,12.707733,0.547591,1.138667,0.034385,0.072,0.0,32.207333,2.172103,0.001667,0.000058


## Latex - Acc

In [130]:
acc, accstr = pd.DataFrame(index=flst, columns=dlst), pd.DataFrame(index=flst, columns=dlst)
for row in acc.index:
    for col in acc.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == col), 'f1micro_test_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == col), 'f1micro_test_std'].values
        if len(mean) > 0:
            accstr.loc[row, col] = f'{mean[0]:.2f}\\tpm{{{std[0]:.2f}}}'
            acc.loc[row, col] = float(mean[0])

accstr

,cora,citeseer,pubmed,flickr,chameleon_filtered,squirrel_filtered,actor,roman_empire
Impulse,86.20\tpm{1.55},75.27\tpm{0.26},83.12\tpm{0.83},23.49\tpm{3.96},36.52\tpm{3.93},34.83\tpm{1.50},25.44\tpm{0.72},28.22\tpm{0.80}
Monomial,88.85\tpm{1.19},77.00\tpm{0.98},89.37\tpm{0.34},37.41\tpm{0.63},32.40\tpm{2.53},34.99\tpm{3.12},28.18\tpm{8.12},64.85\tpm{0.89}
PPR,89.09\tpm{1.82},77.10\tpm{0.46},88.94\tpm{0.52},36.62\tpm{0.44},35.21\tpm{2.34},35.89\tpm{3.94},37.24\tpm{0.84},65.48\tpm{1.26}
HK,86.81\tpm{3.89},77.15\tpm{1.04},89.34\tpm{0.11},36.18\tpm{1.03},32.02\tpm{1.49},32.96\tpm{1.69},35.68\tpm{0.67},64.64\tpm{0.24}
Gaussian,88.36\tpm{1.78},77.15\tpm{0.89},89.03\tpm{0.26},35.64\tpm{1.62},35.21\tpm{1.30},36.19\tpm{1.97},37.76\tpm{0.17},66.41\tpm{1.00}
Var-Monomial,85.46\tpm{5.07},77.25\tpm{0.67},88.38\tpm{0.35},37.22\tpm{0.78},35.21\tpm{2.27},40.99\tpm{1.84},36.51\tpm{0.87},65.67\tpm{2.13}
Horner,87.49\tpm{0.91},75.42\tpm{2.38},89.69\tpm{0.62},35.20\tpm{1.14},35.39\tpm{4.46},32.21\tpm{2.17},37.76\tpm{0.70},67.39\tpm{1.53}
Chebyshev,73.81\tpm{1.26},71.40\tpm{0.54},88.53\tpm{0.52},35.90\tpm{0.33},35.58\tpm{3.19},37.09\tpm{2.49},37.87\tpm{0.76},64.92\tpm{0.57}
ChebInterp,82.13\tpm{4.68},76.49\tpm{0.55},90.03\tpm{0.41},32.24\tpm{3.37},40.82\tpm{1.17},36.71\tpm{1.41},31.93\tpm{7.55},65.63\tpm{0.81}
Clenhaw,86.20\tpm{0.77},72.98\tpm{7.28},78.04\tpm{19.26},28.73\tpm{4.31},30.90\tpm{6.81},32.73\tpm{2.94},38.03\tpm{1.20},65.92\tpm{1.98}


In [131]:
acc.dropna(inplace=True)
acc = acc.astype(float)
accstr = accstr.loc[acc.index]

rk = ['a', 'b', 'c']
for col in acc.columns:
    top3_indices = acc[col].nlargest(3).index
    for i, index in enumerate(top3_indices):
        accstr.loc[index, col] = f'\\rk{rk[i]}' + '{}' + accstr.loc[index, col]

accstr

,cora,citeseer,pubmed,flickr,chameleon_filtered,squirrel_filtered,actor,roman_empire
Impulse,86.20\tpm{1.55},75.27\tpm{0.26},83.12\tpm{0.83},23.49\tpm{3.96},36.52\tpm{3.93},34.83\tpm{1.50},25.44\tpm{0.72},28.22\tpm{0.80}
Monomial,\rkb{}88.85\tpm{1.19},77.00\tpm{0.98},89.37\tpm{0.34},\rkb{}37.41\tpm{0.63},32.40\tpm{2.53},34.99\tpm{3.12},28.18\tpm{8.12},64.85\tpm{0.89}
PPR,\rka{}89.09\tpm{1.82},77.10\tpm{0.46},88.94\tpm{0.52},36.62\tpm{0.44},35.21\tpm{2.34},35.89\tpm{3.94},37.24\tpm{0.84},65.48\tpm{1.26}
HK,86.81\tpm{3.89},77.15\tpm{1.04},89.34\tpm{0.11},36.18\tpm{1.03},32.02\tpm{1.49},32.96\tpm{1.69},35.68\tpm{0.67},64.64\tpm{0.24}
Gaussian,88.36\tpm{1.78},77.15\tpm{0.89},89.03\tpm{0.26},35.64\tpm{1.62},35.21\tpm{1.30},36.19\tpm{1.97},37.76\tpm{0.17},\rkc{}66.41\tpm{1.00}
Var-Monomial,85.46\tpm{5.07},\rkc{}77.25\tpm{0.67},88.38\tpm{0.35},\rkc{}37.22\tpm{0.78},35.21\tpm{2.27},\rka{}40.99\tpm{1.84},36.51\tpm{0.87},65.67\tpm{2.13}
Horner,87.49\tpm{0.91},75.42\tpm{2.38},\rkc{}89.69\tpm{0.62},35.20\tpm{1.14},35.39\tpm{4.46},32.21\tpm{2.17},37.76\tpm{0.70},\rka{}67.39\tpm{1.53}
Chebyshev,73.81\tpm{1.26},71.40\tpm{0.54},88.53\tpm{0.52},35.90\tpm{0.33},35.58\tpm{3.19},37.09\tpm{2.49},37.87\tpm{0.76},64.92\tpm{0.57}
ChebInterp,82.13\tpm{4.68},76.49\tpm{0.55},\rka{}90.03\tpm{0.41},32.24\tpm{3.37},\rka{}40.82\tpm{1.17},36.71\tpm{1.41},31.93\tpm{7.55},65.63\tpm{0.81}
Clenhaw,86.20\tpm{0.77},72.98\tpm{7.28},78.04\tpm{19.26},28.73\tpm{4.31},30.90\tpm{6.81},32.73\tpm{2.94},\rkc{}38.03\tpm{1.20},65.92\tpm{1.98}


In [132]:
for row in accstr.index:
    print("\t& " + row + " & " + " & ".join(accstr.loc[row].values) + " \\\\")

	& Impulse & 86.20\tpm{1.55} & 75.27\tpm{0.26} & 83.12\tpm{0.83} & 23.49\tpm{3.96} & 36.52\tpm{3.93} & 34.83\tpm{1.50} & 25.44\tpm{0.72} & 28.22\tpm{0.80} \\
	& Monomial & \rkb{}88.85\tpm{1.19} & 77.00\tpm{0.98} & 89.37\tpm{0.34} & \rkb{}37.41\tpm{0.63} & 32.40\tpm{2.53} & 34.99\tpm{3.12} & 28.18\tpm{8.12} & 64.85\tpm{0.89} \\
	& PPR & \rka{}89.09\tpm{1.82} & 77.10\tpm{0.46} & 88.94\tpm{0.52} & 36.62\tpm{0.44} & 35.21\tpm{2.34} & 35.89\tpm{3.94} & 37.24\tpm{0.84} & 65.48\tpm{1.26} \\
	& HK & 86.81\tpm{3.89} & 77.15\tpm{1.04} & 89.34\tpm{0.11} & 36.18\tpm{1.03} & 32.02\tpm{1.49} & 32.96\tpm{1.69} & 35.68\tpm{0.67} & 64.64\tpm{0.24} \\
	& Gaussian & 88.36\tpm{1.78} & 77.15\tpm{0.89} & 89.03\tpm{0.26} & 35.64\tpm{1.62} & 35.21\tpm{1.30} & 36.19\tpm{1.97} & 37.76\tpm{0.17} & \rkc{}66.41\tpm{1.00} \\
	& Var-Monomial & 85.46\tpm{5.07} & \rkc{}77.25\tpm{0.67} & 88.38\tpm{0.35} & \rkc{}37.22\tpm{0.78} & 35.21\tpm{2.27} & \rka{}40.99\tpm{1.84} & 36.51\tpm{0.87} & 65.67\tpm{2.13} \\
	& Horner & 